# Train a model with PyTorch

> This notebook is a simplified version of our workflow. It exposes the basic details of the traning and evaluation loop more explicitly, but does not offer advanced features like early stopping, mini-batches or validation. Use the `*-lightning` version for those.

## How to use

Create a copy of this template into a new subdirectory with name `xxx_some-unique-name`, where `xxx` is a 3-digit number. Since the model files are not pushed to GitHub for size reasons, they will stay in your machine. Use a unique name for that subdirectory so people can identify experiments easily. Each subdirectory should ONLY contain a single notebook and featurization exercise. Then, define the hyperparameters below.

In [9]:
# If this is the template file (and not a copy) and you are introducing changes,
# update VERSION with the current date (YYYY.MM.DD)
VERSION = "2020.12.05"

## ✏ Define hyper parameters

Edit `UPPERCASE` variables in the following cell to configure behavior of the training.

In [2]:
# DATA -- Glob paths must be relative to the root of the repository: REPO / features
NPZ_FILES = [
    "_output/ligand__SmilesToLigandFeaturizer_style=rdkit__MorganFingerprintFeaturizer_nbits=512_radius=2/PKIS2DatasetProvider/*.npz",
]

# Model -- specified with the full import path to the class object
MODEL_CLS = "kinoml.ml.torch_models.NeuralNetworkRegression"
MODEL_KWARGS = {"hidden_size": 350}  # input_size is defined dynamically during training
WITH_OBSERVATION_MODEL = True

# Adam
LEARNING_RATE = 0.001
EPSILON = 1e-7
BETAS = 0.9, 0.999

# Trainer
MAX_EPOCHS = 50
N_SPLITS = 5
SHUFFLE_FOLDS = False
VALIDATION = False  # TODO: VALIDATION=True is not implemented yet!
MIN_ITEMS_PER_DATASET = 50  # skip datasets if len(data) < N

# Bootstrapping
N_BOOTSTRAPS = 1
BOOTSTRAP_SAMPLE_RATIO = 1

# Output
VERBOSE = False

⚠ From here on, you should _not_ need to modify anything else 🤞

---

Define key paths for data and outputs:

In [6]:
from pathlib import Path
import time

HERE = Path(_dh[-1])

for parent in HERE.parents:
    if next(parent.glob(".github/"), None):
        REPO = parent
        break

FEATURES_STORE = REPO / "features"
        
OUT = HERE / "_output" / f"{time.time():.0f}"
OUT.mkdir(parents=True, exist_ok=True)

print(f"This notebook:           HERE = ~/{HERE.relative_to(Path.home())}")
print(f"This repo:               REPO = ~/{REPO.relative_to(Path.home())}")
print(f"Features:      FEATURES_STORE = ~/{FEATURES_STORE.relative_to(Path.home())}")
print(f"Outputs in:               OUT = ~/{OUT.relative_to(Path.home())}")

This notebook:           HERE = ~/devel/py/openkinome/experiments-binding-affinity/001_ligand-based
This repo:               REPO = ~/devel/py/openkinome/experiments-binding-affinity
Features:      FEATURES_STORE = ~/devel/py/openkinome/experiments-binding-affinity/features/_output
Outputs in:               OUT = ~/devel/py/openkinome/experiments-binding-affinity/001_ligand-based/_output/1607172329


In [3]:
# Nasty trick: save all-caps local variables (CONSTANTS working as hyperparameters) so far in a dict to save it later
_hparams = {key: value for key, value in locals().items() if key.upper() == key and not key.startswith(("_", "OE_"))}

In [4]:
# TODO: Make all datasets use the same kinase identifiers
ONE_KINASE = {
    "ChEMBLDatasetProvider": "P35968",
    "PKIS2DatasetProvider": "ABL2",
}

In [5]:
from collections import defaultdict
import shutil

from IPython.display import Markdown
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
import pytorch_lightning as pl

from kinoml.utils import seed_everything, import_object
from kinoml.core import measurements as measurement_types
from kinoml.datasets.torch_datasets import XyNpzTorchDataset
from kinoml.core.measurements import null_observation_model

# Fix the seed for reproducible random splits -- otherwise we get mixed train/test groups every time, biasing the model evaluation
seed_everything()

Reporting results at path: /home/jaime/devel/py/openkinome/experiments-binding-affinity/001_ligand-based/MorganFingerprint/LNN/_output/1607170246


## Load featurized data and create observation models

We assume this path structure: `$REPO/features/_output/<FEATURIZATION>/<DATASET>/<GROUP>.npz`

In [6]:
DATASETS = []
MEASUREMENT_TYPES = set()
KINASES = set()
FEATURIZATIONS = set()
for glob in NPZ_FILES:
    for npz in FEATURES_STORE.glob(glob):
        kinase, measurement_type = npz.stem.split("__")
        dataset = npz.parent.name
        featurization = npz.parents[1].name
        
        MEASUREMENT_TYPES.add(measurement_type)
        KINASES.add(kinase)
        FEATURIZATIONS.add(featurization)
        
        ds = XyNpzTorchDataset(npz)
        ds.metadata = {
            "kinase": kinase,
            "measurement_type": measurement_type,
            "dataset": dataset,
            "featurization": featurization
        }
        DATASETS.append(ds)
        if not VALIDATION:
            ds.indices["test"] = np.concatenate([ds.indices["test"], ds.indices["val"]])
            ds.indices["val"] = np.array([])

In [7]:
print("Observed...")
print(" - Measurement types:", len(MEASUREMENT_TYPES), "-->", *MEASUREMENT_TYPES)
print(" - Kinases:", len(KINASES), "-->", *KINASES)

Observed...
 - Measurement types: 1 --> PercentageDisplacementMeasurement
 - Kinases: 403 --> DCAMKL1 CAMK2D MRCKB NIM1 PCTK1 CDK7 LIMK1 FER PRKD2 CDK11 RIOK1 EPHA4 MAP3K2 PIM1 EPHB6 MEK4 BRSK2 TLK2 RET IKK-alpha FLT1 HIPK4 NDR2 CAMKK2 MYLK2 PIK3CG RSK2(Kin.Dom.2-C-terminal) TRKB p38-gamma DAPK1 LCK EPHB1 CDK5 EPHA5 PFTK1 WNK3 PIK3CB FGR PAK2 JAK1(JH2domain-pseudokinase) p38-alpha GRK2 p38-beta PAK7 RSK4(Kin.Dom.2-C-terminal) HCK GAK JAK3(JH1domain-catalytic) ASK2 CAMKK1 LATS1 PLK4 MAPKAPK5 INSRR DMPK CLK1 EGFR MET ULK3 CIT CAMK2B HIPK3 IGF1R BMPR1A MLK2 SYK GRK3 CSNK2A2 AURKB WNK1 CSF1R-autoinhibited MINK CAMK1G MEK1 ERBB3 NEK10 SRMS STK33 PKN1 MERTK CDK2 STK35 GRK1 CDKL2 PIP5K2B ITK SGK2 STK16 NEK4 GSK3A PIK3CA SgK110 ACVR2B ADCK3 TNIK MTOR PRKCQ PHKG2 LZK PIKFYVE IKK-epsilon TAOK3 PCTK3 MLK3 AMPK-alpha1 STK36 SBK1 BUB1 NEK11 MST4 WEE1 MAP3K3 MEK5 FRK ERBB2 SGK ROCK1 TTK ABL2 KIT PLK3 ERN1 MAP4K5 CAMK1D PHKG1 S6K1 RPS6KA5(Kin.Dom.2-C-terminal) MLK1 RSK1(Kin.Dom.1-N-terminal) RSK1(Kin

## Check X duplication

There's a chance we have several measurements per ligand, or, depending on the featurization scheme, even hash collisions... Let's quantify the amount of input tensor duplication we are facing.

In [8]:
for mtype in MEASUREMENT_TYPES:
    display(Markdown(f"#### {mtype}"))
    unique = {}
    for ds in DATASETS:
        if ds.metadata["measurement_type"] == mtype:
            all_ = ds.data_X.shape[0]
            unique_ = np.unique(ds.data_X, axis=0).shape[0]
            unique[ds.metadata["kinase"]] = {"all": all_, "unique": unique_}
    df = pd.DataFrame.from_dict(unique).T
    df["uniqueness"] = df["unique"] / df["all"]
    # This is how you highlight rows in pandas!
    df = df.describe().style.apply(lambda x: ['font-weight: bold' for v in x], subset=pd.IndexSlice[["mean", "std"], :])
    display(df)

#### PercentageDisplacementMeasurement

,all,unique,uniqueness
count,403.000000,403.000000,403.000000
mean,649.801489,637.000000,0.984738
std,71.772757,0.000000,0.040947
min,645.000000,637.000000,0.329199
25%,645.000000,637.000000,0.987597
50%,645.000000,637.000000,0.987597
75%,645.000000,637.000000,0.987597
max,1935.000000,637.000000,0.987597


Now that we have all the data-dependent objects, we can start with the model-specific definitions.

### Training loop

In [9]:
from kinoml.ml.lightning_modules import KFold3Way, KFold
from IPython.display import Markdown
from tqdm.auto import trange, tqdm
from kinoml.ml.torch_models import NeuralNetworkRegression
from ipywidgets import HBox, VBox, Output, HTML
from kinoml.analysis.plots import predicted_vs_observed, performance
from kinoml.utils import fill_until_next_multiple
import pandas as pd
import torch.nn as nn

if VALIDATION:
    kfold = KFold3Way(n_splits=N_SPLITS, shuffle=SHUFFLE_FOLDS)
    ttypes = ["train", "val", "test"]
else:
    kfold = KFold(n_splits=N_SPLITS, shuffle=SHUFFLE_FOLDS)
    ttypes = ["train", "test"]

ModelCls = import_object(MODEL_CLS)
    
kinase_metrics = defaultdict(dict)
for dataset in tqdm(DATASETS):
    kinase = dataset.metadata["kinase"]
    mtype = dataset.metadata["measurement_type"]
    if dataset.data_X.shape[0] < MIN_ITEMS_PER_DATASET:
        print("Ignoring", kinase, "because it has less than", MIN_ITEMS_PER_DATASET, "entries for type", mtype)
        continue
            
    if VERBOSE:
        display(Markdown(f"#### {mtype}"))

    mtype_class = getattr(measurement_types, mtype)
    obs_model = mtype_class.observation_model(backend="pytorch")
    metrics = defaultdict(list)

    for fold_index, splits in enumerate(kfold.split(dataset.data_X, dataset.data_y)):
        if VALIDATION:
            train_indices, val_indices, test_indices = splits
        else:
            train_indices, test_indices = splits

        if VERBOSE:
            display(Markdown(f"##### Fold {fold_index}"))

        ####
        # TRAIN
        ####
        x_train = dataset.data_X[train_indices].float()
        x_test = dataset.data_X[test_indices].float()
        y_train = dataset.data_y[train_indices]
        y_test = dataset.data_y[test_indices]

        if VALIDATION:
            x_val = dataset.data_X[val_indices].float()
            y_val = dataset.data_y[val_indices]

        nn_model = ModelCls(input_size=x_train.shape[1], **MODEL_KWARGS)
        nn_model.train(True)

        optimizer = torch.optim.Adam(nn_model.parameters(), lr=LEARNING_RATE, eps=EPSILON, betas=BETAS)
        loss_function = torch.nn.MSELoss()

        if VERBOSE:
            range_epochs = trange(MAX_EPOCHS, desc="Epochs (+ featurization...)")
        else:
            range_epochs = range(MAX_EPOCHS)
        for epoch in range_epochs:
            optimizer.zero_grad()

            prediction = nn_model(x_train)
            if WITH_OBSERVATION_MODEL:
                prediction = obs_model(prediction)

            prediction = prediction.view_as(y_train)

            # prediction = delta_g
            loss = loss_function(prediction, y_train)
            if VERBOSE:
                range_epochs.set_description(f"Epochs (loss={loss.item():.2e})")

            if VALIDATION:
                raise NotImplementedError("Validation step not implemented yet")


            # Gradients w.r.t. parameters
            loss.backward()

            # Optimizer
            optimizer.step()

        ####
        # EVAL
        ####
        nn_model.eval()
        outputs = []
        for ttype in ttypes:
            output = Output()
            with output:
                title = f"fold={fold_index}, {ttype}={locals()[f'{ttype}_indices'].shape[0]}"
                print(title)
                print("-"*(len(title)))

                observed = locals()[f"y_{ttype}"]

                with torch.no_grad():
                    predicted = nn_model(locals()[f"x_{ttype}"])
                    if WITH_OBSERVATION_MODEL:
                        predicted = obs_model(predicted)

                predicted = predicted.view_as(observed).detach().numpy()
                observed = observed.detach().numpy()
                these_metrics = performance(predicted, observed, n_boot=N_BOOTSTRAPS, sample_ratio=BOOTSTRAP_SAMPLE_RATIO)
                metrics[ttype].append(these_metrics)
                if VERBOSE:
                    display(predicted_vs_observed(predicted, observed, mtype_class, with_metrics=False))

            outputs.append(output)
        if VERBOSE:
            display(HBox(outputs))

    # Average performances

    average = defaultdict(dict)
    for key in metrics["test"][0]:
        for label in ttypes:
            # this zero here ---v is super important! we only want the mean of the means!
            values =  [fold[key][0] for fold in metrics[label]]
            average[label][key] = {
                "mean": np.mean(values),
                "std": np.std(values)
            }
    if VERBOSE:
        for label in ttypes:    
            display(HTML(f"Bootstrapped average across folds ({label}):"))
            display(pd.DataFrame.from_dict(average[label]))
    kinase_metrics[kinase][mtype] = average

### Summary

`kinase_metrics` is a nested dictionary with these dimensions:

- kinase name
- measurement type
- metric
- mean & standard deviation

In [10]:
import json
display(Markdown(f"""
### Configuration 

```json
{json.dumps(_hparams, default=str, indent=2)}
```
"""))
for mtype in MEASUREMENT_TYPES:
    display(Markdown(f"#### {mtype}"))
    # This is going to be fun:
    df = pd.concat({kinase_name: 
                    pd.DataFrame.from_dict(
                        {f"{train_test}_{metric}_{stat}": (value,) 
                         for train_test, vv in v.get(mtype, {}).items() 
                         for metric, vvv    in vv.items() 
                         for stat, value    in vvv.items()}
                    )       
                    for kinase_name, v in sorted(kinase_metrics.items(), key=lambda kv: kv[0].lower())})
    
    df.index = [index[0] for index in df.index]
    with pd.option_context("display.float_format", "{:.3f}".format, "display.max_rows", len(df)):
        display(df.style.background_gradient(subset=["train_r2_mean", "test_r2_mean"], low=0, high=1, vmin=0, vmax=1))
        display(df.describe()[["train_r2_mean", "train_r2_std", "test_r2_mean", "test_r2_std"]].describe().style.apply(lambda x: ['font-weight: bold' for v in x], subset=pd.IndexSlice[["mean", "std"], :]))


### Configuration 

```json
{
  "HERE": "/home/jaime/devel/py/openkinome/experiments-binding-affinity/001_ligand-based/MorganFingerprint",
  "REPO": "/home/jaime/devel/py/openkinome/experiments-binding-affinity",
  "NPZ_FILES": [
    "features/_output/ligand__SmilesToLigandFeaturizer_style=rdkit__MorganFingerprintFeaturizer_nbits=512_radius=2/PKIS2DatasetProvider/*.npz"
  ],
  "MODEL_CLS": "kinoml.ml.torch_models.NeuralNetworkRegression",
  "MODEL_KWARGS": {
    "hidden_size": 350
  },
  "WITH_OBSERVATION_MODEL": true,
  "LEARNING_RATE": 0.001,
  "EPSILON": 1e-07,
  "BETAS": [
    0.9,
    0.999
  ],
  "MAX_EPOCHS": 50,
  "N_SPLITS": 5,
  "SHUFFLE_FOLDS": false,
  "VALIDATION": false,
  "MIN_ITEMS_PER_DATASET": 50,
  "N_BOOTSTRAPS": 1,
  "BOOTSTRAP_SAMPLE_RATIO": 1,
  "VERBOSE": false
}
```


#### PercentageDisplacementMeasurement

,train_mae_mean,train_mae_std,train_mse_mean,train_mse_std,train_r2_mean,train_r2_std,train_rmse_mean,train_rmse_std,test_mae_mean,test_mae_std,test_mse_mean,test_mse_std,test_r2_mean,test_r2_std,test_rmse_mean,test_rmse_std
AAK1,11.555816,0.579438,242.095343,35.163736,0.652107,0.036082,15.518225,1.131379,16.232491,1.352591,476.882434,75.613343,0.207395,0.158547,21.771564,1.697483
ABL1-nonphosphorylated,13.206783,0.453847,304.388318,13.579086,0.688075,0.023088,17.442487,0.384640,20.911985,5.032978,829.181714,349.163354,0.083383,0.177752,28.280630,5.421042
ABL2,9.220394,0.795624,166.666733,28.820420,0.733797,0.054933,12.862127,1.110144,15.827595,2.922895,529.204358,216.425054,0.169368,0.255321,22.517148,4.709822
ACVR1,10.777476,0.525727,234.782422,25.449109,0.395103,0.043724,15.299972,0.832621,13.799110,1.745761,439.704681,111.898823,0.095336,0.096799,20.790980,2.727602
ACVR1B,11.066205,0.394495,248.897284,17.025588,0.536738,0.058444,15.767165,0.542024,13.352255,2.345512,406.917569,114.539151,0.213238,0.118198,19.928618,3.125339
ACVR2A,6.841238,0.733465,271.655576,63.002453,-0.065844,0.018416,16.365551,1.955585,7.606833,1.127197,291.951570,129.095486,-0.118771,0.053398,16.625368,3.943183
ACVR2B,11.228427,0.954196,218.065762,25.662694,0.416434,0.069824,14.741842,0.862464,14.204533,1.254642,404.959906,104.853850,0.138343,0.117667,19.946405,2.664737
ACVRL1,6.118878,0.177871,213.744730,13.209841,-0.069939,0.016812,14.612891,0.456227,7.076820,1.746675,253.076593,125.104158,-0.123878,0.039087,15.432244,3.862960
ADCK3,10.132405,0.538982,164.994525,14.384503,0.302244,0.087443,12.832862,0.558715,12.354020,2.407879,265.851862,120.408856,0.075234,0.130095,15.926293,3.493573
ADCK4,10.050524,0.156375,177.809729,9.900195,0.668736,0.043022,13.329302,0.373409,16.009918,1.768606,429.577075,113.382867,-1.195494,3.105170,20.569881,2.541077


,train_r2_mean,train_r2_std,test_r2_mean,test_r2_std
count,8.000000,8.000000,8.000000,8.000000
mean,50.702726,50.430978,50.307001,50.878660
std,142.349990,142.459411,142.510691,142.282273
min,-0.237049,0.005386,-1.449941,0.015152
25%,0.261390,0.033314,0.014044,0.136965
50%,0.404274,0.050175,0.126411,0.213506
75%,0.708849,0.103233,0.279385,0.953584
max,403.000000,403.000000,403.000000,403.000000


### Save reports to disk

In [11]:
%%capture cap --no-stderr
from kinoml.utils import watermark

w = watermark()

In [12]:
import json

df.to_csv(OUT / "performance.csv")

with open(OUT / "performance.json", "w") as f:
    json.dump(kinase_metrics, f, default=str, indent=2)
    
with open(OUT/ "watermark.txt", "w") as f:
    f.write(cap.stdout)

with open(OUT / "hparams.json", "w") as f:
    json.dump(_hparams, f, default=str, indent=2)